In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium==4.9.1

Executing: /tmp/apt-key-gpghome.hCEFhTWI2L/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.XZxLyQVzOk/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.xxOrcjEve4/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

from selenium.webdriver.remote.remote_connection import LOGGER
#LOGGER.setLevel(logging.WARNING)
#sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("window-size=1900,800")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

In [ ]:
import sys
import logging
from tqdm.notebook import tqdm
import pandas
import json
import pprint
import pandas as pd
import urllib.parse
import re
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Quality of life

In [ ]:
wd = webdriver.Chrome('chromedriver', options=chrome_options)

wd.get("https://www.numbeo.com/quality-of-life/country_result.jsp?country=Italy")
cities = wd.find_elements(By.ID, "city")[0].text.split('\n')[1:]
cities = [i.replace('          ','') for i in cities]

df_all = pd.DataFrame()
failed=[]

def get_data_city(city):

  wd.get(f"https://www.numbeo.com/quality-of-life/city_result.jsp?country=Italy&city={city}")

  i=0
  table = ''
  while 'ƒ' not in table:
    table = wd.find_elements(By.TAG_NAME, "table")[i].text
    i+=1

  list_index=table.replace('Very Low','')\
          .replace('Very High','')\
          .replace('High','')\
          .replace('Moderate','')\
          .replace('Low','')\
          .replace('ƒ','')\
          .replace('   ','').replace('?','').split('\n')

  df= pd.DataFrame(
      [
          [' '.join(j.split(' ')[:-1]),j.split(' ')[-1]]
            for j in list_index],
          columns=['param','value']
      )
  df['city']=city

  return df

for city in cities:
  try:
    df_all = pd.concat([get_data_city(city),df_all])
    print(city)
  except:
    print('!! FAILED: '+city)
    failed.append(city)

In [ ]:
df_all.value=df_all.value.replace('',None).astype(float)
df_all.param=df_all.param.str.replace('  ','').str.replace(':','')
df_all.to_csv('/content/drive/MyDrive/Colab Notebooks/data_project/index.csv',index=False)

## Cost of living

In [ ]:
wd = webdriver.Chrome('chromedriver', options=chrome_options)

In [ ]:
data=wd.find_elements(By.CLASS_NAME, "data_wide_table")[0].text.split('\n')

data_numb = [i for i in data if ((re.search("\d", i) is not None) & ('?' not in i))]



ranges = [i.split(' ')[-1]if '-' in i.split(' ')[-1] else None for i in data_numb ]

values=[]
labels=[]
units=[]

for i in data_numb:

  if ('€' in i):
    units.append('€')
  elif ('$' in i):
    units.append('$')
  else:
    units.append(None)

  split=i.split(' ')
  split.reverse()
  for iter, j in enumerate(split):
    if ('-' in j) | ('€' in j ) | ('$' in j ):
      pass
    else:
      values.append(float(j.replace(',','')))

      break
  label=split[iter+1:]
  label.reverse()
  labels.append(' '.join(label))

df = pd.DataFrame({'param':labels, 'value':values, 'range':ranges, 'unit': units})
df['lower'] = df.range.str.split('-').str[0]
df['upper'] = df.range.str.split('-').str[1]
df.drop(columns='range',inplace=True)

In [ ]:
def get_data_city(city):

  wd.get(f"https://www.numbeo.com/cost-of-living/city_result.jsp?country=Italy&city={city}")

  data=wd.find_elements(By.CLASS_NAME, "data_wide_table")[0].text.split('\n')

  data_numb = [i for i in data if ((re.search("\d", i) is not None) & ('?' not in i))]

  if not data_numb:
    return None

  ranges = [i.split(' ')[-1]if '-' in i.split(' ')[-1] else None for i in data_numb ]

  values=[]
  labels=[]
  units=[]

  for i in data_numb:

    if ('€' in i):
      units.append('€')
    elif ('$' in i):
      units.append('$')
    else:
      units.append(None)

    split=i.split(' ')
    split.reverse()
    for iter, j in enumerate(split):
      if ('-' in j) | ('€' in j ) | ('$' in j ):
        pass
      else:
        values.append(float(j.replace(',','')))

        break
    label=split[iter+1:]
    label.reverse()
    labels.append(' '.join(label))

  df = pd.DataFrame({'param':labels, 'value':values, 'range':ranges, 'unit': units})
  df['lower'] = df.range.str.split('-').str[0]
  df['upper'] = df.range.str.split('-').str[1]
  df.drop(columns='range',inplace=True)
  df['city']=city
  return df

In [ ]:
wd = webdriver.Chrome('chromedriver', options=chrome_options)
df_all = pd.DataFrame()
failed=[]

for city in cities[:]:
  try:
    df_all = pd.concat([get_data_city(city),df_all])
    print(city)
  except:
    print('!! FAILED: '+city)
    failed.append(city)

In [ ]:
df_all.to_csv('/content/drive/MyDrive/Colab Notebooks/data_project/cost_of_living.csv',index=False)

## Coordinates

In [ ]:
import json
import pandas as pd
import requests
from tqdm import tqdm
index = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_project/cost_of_living.csv',index_col=0)
coordinates = pd.DataFrame(columns=['name_complete','latitude','longitude'])
failed=[]
for city in tqdm(index.city.unique()):
  try:
    #print(city)
    r = requests.get(f'https://nominatim.openstreetmap.org/search?q={city}&limit=2&format=json')
    info = json.loads(r.text)[0]
    coordinates.loc[city] = [info.get('display_name'), info.get('lat'), info.get('lon')]
  except:
    print(f'FAILED {city}')
    failed.append(city)
print(failed)


100%|██████████| 222/222 [02:46<00:00,  1.34it/s]

[]


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_project/index.csv',index_col=0)
df=df.reset_index().merge(coordinates,on='city')
#df['city']=df.name_complete.str.split(',').str[0]
#df['province']=df.name_complete.str.split(',').str[-4]
#df['region']=df.name_complete.str.split(',').str[-3]
#df['cap']=df.name_complete.str.split(',').str[3]
df.drop(columns='name_complete', inplace=True)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/data_project/index_2.csv',index=False)
df.head()

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_project/cost_of_living.csv',index_col=0)
df=df.merge(coordinates,on='city')
#df['city']=df.name_complete.str.split(',').str[0]
#df['province']=df.name_complete.str.split(',').str[-4]
#df['region']=df.name_complete.str.split(',').str[-3]
#df['cap']=df.name_complete.str.split(',').str[3]
df.drop(columns='name_complete', inplace=True)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/data_project/cost_of_living_2.csv',index=False)
df.head()

,param,value,unit,lower,upper,city,latitude,longitude
0,"Meal, Inexpensive Restaurant",11.00,€,10,30,Zambrone,38.69918,15.990094
1,"Meal for 2 People, Mid-range Restaurant, Three...",65.00,€,60,70,Zambrone,38.69918,15.990094
2,McMeal at McDonalds (or Equivalent Combo Meal),11.75,€,8.5,15,Zambrone,38.69918,15.990094
3,Domestic Beer (1 pint draught),3.50,€,3.5,10,Zambrone,38.69918,15.990094
4,Imported Beer (12 oz small bottle),4.50,€,3,6,Zambrone,38.69918,15.990094


formato kepler

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_project/cost_of_living_2.csv',index_col=0)
df.pivot_table(index='city',columns='param',values='value').merge(\
  df.pivot_table(index='city',values=['latitude','longitude']), on ='city').to_csv('/content/drive/MyDrive/Colab Notebooks/data_project/costofliving_kepler.csv')


## Personalized quality index

In [ ]:
costofliv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_project/cost_of_living.csv',index_col=0)
index = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_project/index.csv',index_col=0)
params = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data_project/Numbeo_Calculation_v3.xlsx',sheet_name='Sheet5',skiprows=15)

In [ ]:
def calculate_index(city):
  basketNY=52546.08
  salaryNY=6419.42
  costofliv_city=costofliv[costofliv.city==city].copy()
  index_city=index[index.city==city]
  costofliv_city['param'].replace('Domestic Beer (1 pint draught)','Domestic Beer (0.5 liter draught)',inplace=True)
  merge_city=params.dropna(subset='Unnamed: 2').merge(costofliv_city, left_on='Unnamed: 2',right_on='param',how='left').dropna(subset='param')
  merge_city.head()

  basketConsPlusRent_list=np.array([(merge_city.value*merge_city[i]*merge_city.Param).sum() for i in ['con 1 household member', 'con 2 household member', 'con 3 household member',	'Parameter Standard 4 pers'	]])
  arr = np.array(
        [[[city,
          salary,
          member+1,
          basketConsPlusRent,
          salary / basketConsPlusRent/ (basketNY/salaryNY)*100,
          max(0, 100 + salary / basketConsPlusRent *10*100 / 2.5 - (index_city.loc['Property Price to Income Ratio','value'] * 1.0) - index_city.loc['Cost of Living Index','value'] / 10 + index_city.loc['Safety Index','value'] / 2.0 + index_city.loc['Health Care Index','value'] / 2.5 - index_city.loc['Traffic Commute Time Index','value'] / 2.0 - index_city.loc['Pollution Index','value'] * 2.0 / 3.0 + index_city.loc['Climate Index','value'] / 3.0)]
      for salary in [1000,1300,1500,1748.23,1800,2000,2500,3000,3500]]
        for member,basketConsPlusRent in enumerate(basketConsPlusRent_list)]
    )
  return pd.DataFrame(arr.reshape(-1, arr.shape[-1]), columns=['city', 'salary', 'members','basketConsPlusRent','LocalPuchasingPowerIndex','QualityIndex'])
pers_index=pd.concat([calculate_index(city) for city in index.loc[' Quality of Life Index'].dropna().city.unique()])
pers_index[['salary','basketConsPlusRent','QualityIndex','LocalPuchasingPowerIndex']]=pers_index[['salary','basketConsPlusRent','QualityIndex','LocalPuchasingPowerIndex']].astype(float).round(1)
pers_index[['members']]=pers_index[['members']].astype(int)
pers_index.to_csv('/content/drive/MyDrive/Colab Notebooks/data_project/pers_index.csv',index=False)
